In [1]:
#!pip install dask-kubernetes

In [2]:
from dask_kubernetes import KubeCluster

In [3]:
spec = {
    'kind': 'Pod',
    'metadata': {
        'labels': {
            'foo': 'bar'
        }
    },
    'spec': {
        'restartPolicy': 'Never',
        'containers': [
            {
                'image': 'daskdev/dask:latest',
                'imagePullPolicy': 'IfNotPresent',
                'args': [
                    'dask-worker', '--nthreads', '2', '--no-dashboard', 
                    '--memory-limit', '6GB', '--death-timeout', '60'
                ],
                'name': 'dask',
                'env': [
                    {
                        'name': 'EXTRA_PIP_PACKAGES',  
                         'value': 'git+https://github.com/dask/distributed'
                    }
                ],
                'resources': {
                    'limits': {'cpu': 2, 'memory': '6G'},
                    'requests': {'cpu': 2, 'memory': '6G'}
                }
            }
        ]
    }
}

In [4]:
# c = KubeCluster.from_yaml('./workers.yaml', namespace='dask-ns')
c = KubeCluster.from_dict(spec, namespace='dask-ns')
c

/opt/conda/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38213 instead
  warnings.warn(
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:    tcp://10.1.0.254:40973
distributed.scheduler - INFO -   dashboard at:                    :38213


In [5]:
c.scale(2)

In [6]:
!netstat -an

/bin/bash: netstat: command not found


distributed.scheduler - INFO - Register worker <Worker 'tcp://10.1.1.1:36075', name: 1, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.1.1.1:36075
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.1.1.2:33047', name: 0, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.1.1.2:33047
distributed.core - INFO - Starting established connection


In [7]:
c.close()


distributed.scheduler - INFO - Scheduler closing...
distributed.scheduler - INFO - Scheduler closing all comms
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.1.1.1:36075', name: 1, memory: 0, processing: 0>
distributed.core - INFO - Removing comms to tcp://10.1.1.1:36075
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.1.1.2:33047', name: 0, memory: 0, processing: 0>
distributed.core - INFO - Removing comms to tcp://10.1.1.2:33047
distributed.scheduler - INFO - Lost all workers
